# Bar Menu Transformer - "Hard to Explain" Bar in NYC

In [1]:
# Import Python Libraries
import requests                 # Make http requests (get website data)
import pandas as pd             # DataFrames, Excel 
from bs4 import BeautifulSoup   # Website Scraping
from datetime import datetime   # Calendar Dates

# Constants
URL = "https://www.hardtoexplain.co/menu"

In [2]:
# Function: Get Website Content
def fetch_web_content(url):
    """Fetch content from a web page."""
    response = requests.get(url)
    return response.content

In [3]:
def parse_menu(content):
    """Parse the menu data from the web content using BeautifulSoup."""
    data = []
    soup = BeautifulSoup(content, 'html.parser')
    
    # Find the main menu container
    menus_div = soup.find('div', class_='menus')
    
    # Iterate over each menu in the container
    for menu_div in menus_div.find_all('div', class_="menu"):
        menu_label = menu_div['aria-label']
        
        # Iterate over each section in a menu
        for menu_section_div in menu_div.find_all('div', class_="menu-section"):
            section_header = menu_section_div.find('div', class_="menu-section-title").text
            
            # Iterate over each item in a section
            for menu_item_div in menu_section_div.find_all('div', class_='menu-item'):
                name = menu_item_div.find('div', class_='menu-item-title').text

                try:
                    price = menu_item_div.find('span', class_='menu-item-price-top').text
                    # Clean up the price format
                    price = price.strip().replace('\n', ' ').replace('\r', '').replace('  ', '').replace('/', ' / ')
                except AttributeError:
                    price = ""

                try:
                    description = menu_item_div.find('div', class_='menu-item-description').text
                except AttributeError:
                    description = ""
                
                # Get the item options, if present
                options = []
                menu_item_options_div = menu_item_div.find('div', class_='menu-item-options')
                if menu_item_options_div:
                    for menu_item_option_div in menu_item_options_div.find_all('div', class_='menu-item-option'):
                        options.append(menu_item_option_div.text.strip())
                
                data.append({
                    'Type': menu_label,
                    'Section': section_header,
                    'Name': name,
                    'Price': price,
                    'Description': description,
                    'Options': '\n'.join(options)
                })

    return pd.DataFrame(data)

In [4]:
# Function: Save Dataframe to Excel and format
def save_to_excel(df, filename):
    """Save the DataFrame to Excel with formatting."""
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        # Write the dataframe to excel
        df.to_excel(writer, sheet_name='Sheet1', index=False)
        
        # Access the xlsxwriter objects
        workbook  = writer.book
        worksheet = writer.sheets['Sheet1']
        
        # Define formats
        bold_format = workbook.add_format({'bold': True})
        word_wrap_format = workbook.add_format({'text_wrap': True, 'valign': 'top'})
        
        # Apply bold format to headers
        for col_num, value in enumerate(df.columns.values):
            worksheet.write(0, col_num, value, bold_format)
            
        # Apply word wrap and conditional formatting to data
        (max_row, max_col) = df.shape
        for row in range(1, max_row + 1):
            for col in range(max_col):
                cell_value = df.iloc[row-1, col]
                
                # Set the Type and Section values to blank if they're the same as the previous row
                if col in [0, 1] and row > 1 and cell_value == df.iloc[row-2, col]:
                    cell_value = ""
                
                worksheet.write(row, col, cell_value, word_wrap_format)
        
        # Set column widths
        column_widths = [1, 2, 3, 1, 3, 8]
        for i, width in enumerate(column_widths):
            worksheet.set_column(i, i, width*10)

In [6]:
# Main script execution
content = fetch_web_content(URL)
df = parse_menu(content)
timestamp = datetime.today().strftime('%Y%m%d%H%M')
filename = f"output/HTE_Menu_{timestamp}.xlsx"
save_to_excel(df, filename)
print(f"{filename} has been created with word wrap set for all cells.")

AttributeError: 'NoneType' object has no attribute 'text'